> # Day 13: Point of Incidence
>
>With your help, the hot springs team locates an appropriate spring which launches you neatly and precisely up to the edge of Lava Island.
>
>There's just one problem: you don't see any lava.
>
>You do see a lot of ash and igneous rock; there are even what look like gray mountains scattered around. After a while, you make your way to a nearby cluster of mountains only to discover that the valley between them is completely full of large mirrors. Most of the mirrors seem to be aligned in a consistent way; perhaps you should head in that direction?
>
>As you move through the valley of mirrors, you find that several of them have fallen from the large metal frames keeping them in place. The mirrors are extremely flat and shiny, and many of the fallen mirrors have lodged into the ash at strange angles. Because the terrain is all one color, it's hard to tell where it's safe to walk or where you're about to run into a mirror.
>
>You note down the patterns of ash (.) and rocks (#) that you see as you walk (your puzzle input); perhaps by carefully analyzing these patterns, you can figure out where the mirrors are!
>
>For example:
>```
>#.##..##.
>..#.##.#.
>##......#
>##......#
>..#.##.#.
>..##..##.
>#.#.##.#.
>
>#...##..#
>#....#..#
>..##..###
>#####.##.
>#####.##.
>..##..###
>#....#..#
>```
>To find the reflection in each pattern, you need to find a perfect reflection across either a horizontal line between two rows or across a vertical line between two columns.
>
>In the first pattern, the reflection is across a vertical line between two columns; arrows on each of the two columns point at the line between the columns:
>```
>123456789
>    ><   
>#.##..##.
>..#.##.#.
>##......#
>##......#
>..#.##.#.
>..##..##.
>#.#.##.#.
>    ><   
>123456789
>```
>In this pattern, the line of reflection is the vertical line between columns 5 and 6. Because the vertical line is not perfectly in the middle of the pattern, part of the pattern (column 1) has nowhere to reflect onto and can be ignored; every other column has a reflected column within the pattern and must match exactly: column 2 matches column 9, column 3 matches 8, 4 matches 7, and 5 matches 6.
>
>The second pattern reflects across a horizontal line instead:
>```
>1 #...##..# 1
>2 #....#..# 2
>3 ..##..### 3
>4v#####.##.v4
>5^#####.##.^5
>6 ..##..### 6
>7 #....#..# 7
>```
>This pattern reflects across the horizontal line between rows 4 and 5. Row 1 would reflect with a hypothetical row 8, but since that's not in the pattern, row 1 doesn't need to match anything. The remaining rows match: row 2 matches row 7, row 3 matches row 6, and row 4 matches row 5.
>
>To summarize your pattern notes, add up the number of columns to the left of each vertical line of reflection; to that, also add 100 multiplied by the number of rows above each horizontal line of reflection. In the above example, the first pattern's vertical line has 5 columns to its left and the second pattern's horizontal line has 4 rows above it, a total of 405.
>
>Find the line of reflection in each of the patterns in your notes. What number do you get after summarizing all of your notes?


## Read input

Since we're comparing individual rows or columns to each other and we're not moving around in the grid, we don't need to use a coordinate system for this puzzle. I split each grid by a new line to rows of strings.

In [1]:
from utils import read_multisection_input


def transformer(section):
    return section.split('\n')

grids = read_multisection_input(13, [transformer])

I have two helper functions to make the main logic read nicer.

To see if any given line `y` is the lower half of a mirror line, we start with that line and the one above it and compare them to each other. Then we move up and down simulatenously and keep checking until either we find two lines that don't match (meaning original was not a mirror line) or we run out of the grid in which case it was a mirror line.

We do the same with column `x` but this time we pass the grid in rotated/transposed way.

In [2]:
def is_horisontal_mirror(grid, row):
    up = row-1
    down = row
    
    while up >= 0 and down < len(grid):
        if grid[up] != grid[down]:
            return False
        up -= 1
        down += 1
    return True

def is_vertical_mirror(grid, column):
    left = column-1
    right = column
    
    while left >= 0 and right < len(grid):
        if grid[left] != grid[right]:
            return False
        left -= 1
        right += 1
    return True

A handy `zip` trick is used here today. If you want to transpose (turn rows into columns and columns into rows) a matrix or list of lists, you can pass `*grid` into `zip` to make it happen.

Other than that, we go through line by line and then column by column until we find a line that is a mirror line. If it's a horisontal line, we return `100 * y` and if it's a vertical line, we return `x`.

In [3]:
def find_reflection_point(grid):
    for y in range(1, len(grid)):
        if is_horisontal_mirror(grid, y):
            return 100 * y
    
    transposed = list(zip(*grid))
    for x in range(1, len(transposed)):
        if is_vertical_mirror(transposed, x):
            return x

    return None

In [4]:
part_1 = 0
reflection_points = {}
for i, grid in enumerate(grids):
    reflection_point = find_reflection_point(grid)
    reflection_points[i] = reflection_point
    part_1 += reflection_point

print(f'Solution: {part_1}')
assert part_1 == 33122

Solution: 33122


## Part 2

>You resume walking through the valley of mirrors and - SMACK! - run directly into one. Hopefully nobody was watching, because that must have been pretty embarrassing.
>
>Upon closer inspection, you discover that every mirror has exactly one smudge: exactly one . or # should be the opposite type.
>
>In each pattern, you'll need to locate and fix the smudge that causes a different reflection line to be valid. (The old reflection line won't necessarily continue being valid after the smudge is fixed.)
>
>Here's the above example again:
>```
>#.##..##.
>..#.##.#.
>##......#
>##......#
>..#.##.#.
>..##..##.
>#.#.##.#.
>
>#...##..#
>#....#..#
>..##..###
>#####.##.
>#####.##.
>..##..###
>#....#..#
>```
>The first pattern's smudge is in the top-left corner. If the top-left # were instead ., it would have a different, horizontal line of reflection:
>```
>1 ..##..##. 1
>2 ..#.##.#. 2
>3v##......#v3
>4^##......#^4
>5 ..#.##.#. 5
>6 ..##..##. 6
>7 #.#.##.#. 7
>```
>With the smudge in the top-left corner repaired, a new horizontal line of reflection between rows 3 and 4 now exists. Row 7 has no corresponding reflected row and can be ignored, but every other row matches exactly: row 1 matches row 6, row 2 matches row 5, and row 3 matches row 4.
>
>In the second pattern, the smudge can be fixed by changing the fifth symbol on row 2 from . to #:
>```
>1v#...##..#v1
>2^#...##..#^2
>3 ..##..### 3
>4 #####.##. 4
>5 #####.##. 5
>6 ..##..### 6
>7 #....#..# 7
>```
>Now, the pattern has a different horizontal line of reflection between rows 1 and 2.
>
>Summarize your notes as before, but instead use the new different reflection lines. In this example, the first pattern's new horizontal line has 3 rows above it and the second pattern's new horizontal line has 1 row above it, summarizing to the value 400.
>
>In each pattern, fix the smudge and find the different line of reflection. What number do you get after summarizing the new reflection line in each pattern in your notes?

The second part had me puzzled for quite a while for such a relatively straight-forward change. It turned out, I originally was skipping grid change if it found the original mirror line. Sometimes, this was possible to happen though even in cases where there was a new mirror line as well.

That's why I had to add a `disallowed_value` which was stored in part 1, to keep running when it was encountered.

To flip a single coordinate, I find the correct row and slice the string to create a new one, with the required one changed. I also make a copy of the grid when flipping so I don't have to flip it back on the next round.

In [5]:
def flip(y, x, grid):
    g = grid[:]
    to_change = g[y]
    new_character = '.' if to_change[x] == '#' else '#'
    new_line = to_change[:x] + new_character  + to_change[x+1:]
    g[y] = new_line
    return g

In [6]:
def find_reflection_point_part_2(grid, disallowed_value):
    for y in range(1, len(grid)):
        if is_horisontal_mirror(grid, y):
            if y * 100 == disallowed_value:
                continue
            return 100 * y
    
    transposed = list(zip(*grid))
    for x in range(1, len(transposed)):
        if is_vertical_mirror(transposed, x):
            if x == disallowed_value:
                continue
            return x

    return None

The final solution is essentially the same as the first part with two changes:

1. I now loop over all the coordinates and flip them one-by-one
2. I check if this change created a new reflection line or not

I put this code into its own function so I don't need to break out from nested for loops.

In [7]:
def flip_until_found(grid, disallowed_value):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            new_grid = flip(y, x, grid)
            potential_reflection_point = find_reflection_point_part_2(new_grid, disallowed_value)
            if potential_reflection_point is not None:
                return potential_reflection_point

In [8]:
part_2 = 0
for i, grid in enumerate(grids):
    part_2 += flip_until_found(grid, reflection_points[i])

print(f'Solution: {part_2}')
assert part_2 == 32312

Solution: 32312


## Two stars

With 26 out of 50 stars, we're officially over the midway point to Christmas!